In [ ]:
print("\n" + "=" * 70)
print("RESUMEN FINAL Y RECOMENDACIONES")
print("=" * 70)

print("\n✅ WORKFLOW COMPLETADO:\n")

workflow_status = [
    ("✅ Estructura validada", True),
    ("✅ Configuración cargada", True),
    ("✅ Datos generados", 'data_generation' in metrics_log['phases']),
    ("✅ Modelo entrenado", 'training' in metrics_log['phases']),
    ("✅ Evaluación completada", 'evaluation' in metrics_log['phases']),
    ("✅ Visualizaciones generadas", 'visualization' in metrics_log['phases']),
    ("✅ Métricas persistidas", 'metrics' in str(METRICS_DIR.glob('*'))),
]

for task, done in workflow_status:
    symbol = task[:2]  # Usar el símbolo del comienzo
    print(f"  {symbol} {task[2:]}")

print("\n📊 ARCHIVOS GENERADOS:\n")
print(f"  • Métricas: {METRICS_DIR}")
print(f"  • Modelos: {MODELS_DIR}")
print(f"  • Logs: {LOGS_DIR}")
print(f"  • Datos: {DATA_DIR}")

print("\n🎯 PRÓXIMOS PASOS:\n")
print("  1. Revisar métricas en: " + str(METRICS_DIR))
print("  2. Analizar modelos en: " + str(MODELS_DIR))
print("  3. Ejecutar visualizaciones nuevamente si es necesario")
print("  4. Para deployment completo, ejecutar:")
print("     python ../scripts/deploy_complete.py")

print("\n" + "=" * 70)
print("📍 EJECUCIÓN COMPLETADA")
print("=" * 70)

## 9️⃣ Resumen Final y Recomendaciones

In [ ]:
print("\n" + "=" * 70)
print("INVENTARIO DE ARTIFACTS")
print("=" * 70)

def count_files(directory, pattern='*'):
    """Contar archivos en directorio"""
    if not directory.exists():
        return 0
    return len(list(directory.glob(pattern)))

artifact_info = {
    'Models': (MODELS_DIR, '*.zip'),
    'Data': (DATA_DIR, '*.csv'),
    'Metrics': (METRICS_DIR, '*.json'),
    'Logs': (LOGS_DIR, '*.log'),
    'Visualizations': (LOGS_DIR, '*.png'),
}

print("\n📦 ARTIFACTS GENERADOS:\n")

total_artifacts = 0
for category, (path, pattern) in artifact_info.items():
    count = count_files(path, pattern)
    total_artifacts += count
    symbol = "✅" if count > 0 else "⏳"
    print(f"{symbol} {category:20s}: {count:3d} archivo(s)")

print(f"\n📊 Total de artifacts: {total_artifacts}")

# Mostrar contenido de directorios
print("\n" + "=" * 70)
print("CONTENIDO DE DIRECTORIOS CLAVE")
print("=" * 70)

for dir_name, dir_path in [('Models', MODELS_DIR), ('Logs', LOGS_DIR), ('Data', DATA_DIR)]:
    if dir_path.exists():
        files = list(dir_path.glob('*'))[:5]  # Primeros 5
        print(f"\n📁 {dir_name}:")
        for f in files:
            size_mb = f.stat().st_size / (1024*1024) if f.is_file() else 0
            print(f"   └─ {f.name} ({size_mb:.2f} MB)")
        
        if len(list(dir_path.glob('*'))) > 5:
            remaining = len(list(dir_path.glob('*'))) - 5
            print(f"   └─ ... y {remaining} archivo(s) más")

## 8️⃣ Inventario de Artifacts

In [ ]:
print("\n" + "=" * 70)
print("HISTORIAL DE EJECUCIONES")
print("=" * 70)

# Buscar archivos de métricas
metrics_files = sorted(glob.glob(str(METRICS_DIR / 'metrics_*.json')))

if metrics_files:
    print(f"\n📋 Encontradas {len(metrics_files)} ejecuciones anteriores:\n")
    
    execution_history = []
    for json_file in metrics_files[-10:]:  # Últimas 10
        with open(json_file, 'r') as f:
            data = json.load(f)
            
        timestamp = data.get('timestamp', 'Unknown')
        phases_success = sum(1 for p in data.get('phases', {}).values() if isinstance(p, dict) and p.get('status') == 'success')
        phases_total = len(data.get('phases', {}))
        
        execution_history.append({
            'Timestamp': timestamp,
            'Fases OK': f"{phases_success}/{phases_total}",
            'Archivo': Path(json_file).name
        })
    
    if execution_history:
        df_history = pd.DataFrame(execution_history)
        print(df_history.to_string(index=False))
else:
    print("\n📋 No hay ejecuciones anteriores registradas")

## 7️⃣ Historial de Ejecuciones

In [ ]:
if 'session' in locals():
    print("\n" + "=" * 70)
    print("ANÁLISIS DE DATOS DE TELEMETRÍA")
    print("=" * 70)
    
    telemetry = session.telemetry_df
    
    # Estadísticas generales
    print("\n📊 Velocidad (km/h):")
    print(f"  Mean: {telemetry['speed_kmh'].mean():.2f}")
    print(f"  Max: {telemetry['speed_kmh'].max():.2f}")
    print(f"  Std: {telemetry['speed_kmh'].std():.2f}")
    
    print("\n📊 G-Force:")
    print(f"  Mean: {telemetry['g_force'].mean():.2f}")
    print(f"  Max: {telemetry['g_force'].max():.2f}")
    print(f"  Std: {telemetry['g_force'].std():.2f}")
    
    print("\n📊 Lean Angle (grados):")
    print(f"  Mean: {telemetry['lean_angle_deg'].mean():.2f}")
    print(f"  Max: {telemetry['lean_angle_deg'].max():.2f}")
    print(f"  Std: {telemetry['lean_angle_deg'].std():.2f}")
    
    print("\n📊 Fatigue:")
    print(f"  Mean: {telemetry['fatigue'].mean():.3f}")
    print(f"  Max: {telemetry['fatigue'].max():.3f}")
    
    # Crear visualización de distribuciones
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle('Distribuciones de Telemetría', fontsize=14, fontweight='bold')
    
    axes[0, 0].hist(telemetry['speed_kmh'], bins=30, alpha=0.7, color='blue')
    axes[0, 0].set_title('Velocidad (km/h)')
    axes[0, 0].set_xlabel('km/h')
    
    axes[0, 1].hist(telemetry['g_force'], bins=30, alpha=0.7, color='green')
    axes[0, 1].set_title('G-Force')
    axes[0, 1].set_xlabel('G')
    
    axes[1, 0].hist(telemetry['lean_angle_deg'], bins=30, alpha=0.7, color='red')
    axes[1, 0].set_title('Lean Angle (grados)')
    axes[1, 0].set_xlabel('Grados')
    
    axes[1, 1].hist(telemetry['fatigue'], bins=30, alpha=0.7, color='orange')
    axes[1, 1].set_title('Fatiga')
    axes[1, 1].set_xlabel('Fatiga')
    
    plt.tight_layout()
    plt.savefig(LOGS_DIR / 'telemetry_distributions.png', dpi=150, bbox_inches='tight')
    print(f"\n✅ Gráfico de distribuciones guardado")
    plt.show()

## 6️⃣ Estadísticas y Análisis de Datos

In [ ]:
print("\n" + "=" * 70)
print("FASE 6: PERSISTENCIA DE MÉTRICAS")
print("=" * 70)

# Guardar métricas en JSON
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# JSON completo
json_file = METRICS_DIR / f'metrics_{timestamp}.json'
with open(json_file, 'w') as f:
    json.dump(metrics_log, f, indent=2)
print(f"✅ Métricas JSON: {json_file}")

# CSV para análisis
csv_file = METRICS_DIR / f'metrics_summary_{timestamp}.csv'
csv_rows = []
for phase, data in metrics_log['phases'].items():
    row = {'phase': phase, 'timestamp': metrics_log['timestamp']}
    if isinstance(data, dict):
        row.update(data)
    csv_rows.append(row)

df_metrics = pd.DataFrame(csv_rows)
df_metrics.to_csv(csv_file, index=False)
print(f"✅ Métricas CSV: {csv_file}")

# Resumen en texto
txt_file = METRICS_DIR / f'summary_{timestamp}.txt'
with open(txt_file, 'w') as f:
    f.write("=" * 70 + "\n")
    f.write("BIO-ADAPTIVE HAPTIC COACHING SYSTEM - EXECUTION SUMMARY\n")
    f.write("=" * 70 + "\n\n")
    f.write(f"Timestamp: {metrics_log['timestamp']}\n")
    f.write(f"Project: {PROJECT_ROOT}\n\n")
    
    for phase, data in metrics_log['phases'].items():
        f.write(f"\n[{phase.upper()}]\n")
        f.write("-" * 70 + "\n")
        if isinstance(data, dict):
            for key, value in data.items():
                if isinstance(value, float):
                    f.write(f"  {key}: {value:.4f}\n")
                else:
                    f.write(f"  {key}: {value}\n")
        else:
            f.write(f"  {data}\n")

print(f"✅ Resumen TXT: {txt_file}")

# Mostrar resumen
print("\n📊 RESUMEN DE EJECUCIÓN:")
print(df_metrics.to_string(index=False))

## 5️⃣ Persistir Métricas

In [ ]:
# FASE 5: Visualización
print("\n" + "=" * 70)
print("FASE 5: VISUALIZACIÓN DE RESULTADOS")
print("=" * 70)

try:
    from visualize import create_visualization, visualize_training_metrics
    
    print(f"\n⏳ Generando visualizaciones...")
    
    start = time.time()
    
    # Crear figura de visualización principal
    if 'session' in locals() and 'eval_metrics' in locals():
        fig = create_visualization(
            session=session,
            eval_metrics=eval_metrics,
            save_path=LOGS_DIR / 'results_dashboard.png'
        )
        print(f"✅ Dashboard guardado: {LOGS_DIR / 'results_dashboard.png'}")
    
    elapsed = time.time() - start
    
    metrics_log['phases']['visualization'] = {
        'duration_seconds': elapsed,
        'figures_generated': 1,
        'status': 'success'
    }
    
    plt.show()
    print(f"✅ Visualizaciones completadas en {elapsed:.2f}s")
    
except Exception as e:
    print(f"⚠️ Error en visualización: {e}")
    metrics_log['phases']['visualization'] = {'status': 'warning', 'error': str(e)}

In [ ]:
# FASE 4: Evaluación
print("\n" + "=" * 70)
print("FASE 4: EVALUACIÓN DEL MODELO")
print("=" * 70)

try:
    from evaluate import evaluate_trained_model
    
    print(f"\n⏳ Evaluando modelo entrenado...")
    
    if 'model' in locals():
        start = time.time()
        eval_metrics = evaluate_trained_model(
            model=model,
            num_episodes=3,
            render=False
        )
        elapsed = time.time() - start
        
        metrics_log['phases']['evaluation'] = {
            'duration_seconds': elapsed,
            'episodes': 3,
            'mean_episode_reward': eval_metrics.get('mean_reward', 0),
            'mean_episode_length': eval_metrics.get('mean_length', 0),
            'status': 'success'
        }
        
        print(f"✅ Evaluación completada en {elapsed:.2f}s")
        print(f"   📊 Mean Reward: {eval_metrics.get('mean_reward', 0):.2f}")
        print(f"   📊 Mean Length: {eval_metrics.get('mean_length', 0):.0f} steps")
    else:
        print("⚠️ Modelo no disponible, saltando evaluación")
        metrics_log['phases']['evaluation'] = {'status': 'skipped'}
        
except Exception as e:
    print(f"⚠️ Error en evaluación: {e}")
    metrics_log['phases']['evaluation'] = {'status': 'failed', 'error': str(e)}

## 4️⃣ Evaluación y Visualización

In [ ]:
# FASE 3: Entrenamiento
print("\n" + "=" * 70)
print("FASE 3: ENTRENAMIENTO PPO")
print("=" * 70)

try:
    from train import create_training_environment, train_ppo_agent
    
    start = time.time()
    print(f"\n⏳ Creando entorno de entrenamiento...")
    
    train_env, _ = create_training_environment(n_laps=5, num_envs=1)
    
    print(f"⏳ Entrenando modelo PPO (5000 timesteps)...")
    model, train_metrics = train_ppo_agent(
        env=train_env,
        total_timesteps=5000,
        save_dir=MODELS_DIR
    )
    
    elapsed = time.time() - start
    
    metrics_log['phases']['training'] = {
        'duration_seconds': elapsed,
        'timesteps': 5000,
        'mean_reward': train_metrics.get('mean_reward', 0),
        'max_reward': train_metrics.get('max_reward', 0),
        'min_reward': train_metrics.get('min_reward', 0),
        'status': 'success'
    }
    
    print(f"✅ Modelo entrenado en {elapsed:.2f}s")
    print(f"   📈 Mean Reward: {train_metrics.get('mean_reward', 0):.2f}")
    print(f"   📈 Max Reward: {train_metrics.get('max_reward', 0):.2f}")
    
except Exception as e:
    print(f"⚠️ Error en entrenamiento: {e}")
    print("   (Continuando con evaluación)")
    metrics_log['phases']['training'] = {'status': 'warning', 'error': str(e)}

In [ ]:
# FASE 2: Entorno
print("\n" + "=" * 70)
print("FASE 2: ENTORNO GYMNASIUM")
print("=" * 70)

try:
    from environment import MotoBioEnv
    
    print(f"\n⏳ Creando entorno MotoBioEnv...")
    
    env = MotoBioEnv(telemetry_df=session.telemetry_df)
    obs, info = env.reset()
    
    # Probar algunos pasos
    total_reward = 0
    bio_gates = 0
    for step in range(50):
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        if info.get('was_bio_gated', False):
            bio_gates += 1
        if terminated or truncated:
            break
    
    metrics_log['phases']['environment'] = {
        'state_space': str(obs.shape),
        'action_space': env.action_space.n,
        'test_steps': step + 1,
        'total_reward': total_reward,
        'bio_gate_activations': bio_gates,
        'status': 'success'
    }
    
    print(f"✅ Entorno creado")
    print(f"   📐 Estado: {obs.shape}")
    print(f"   🎮 Acciones: {env.action_space.n}")
    print(f"   🚨 Bio-Gates: {bio_gates}")
    
except Exception as e:
    print(f"❌ Error en entorno: {e}")
    traceback.print_exc()
    metrics_log['phases']['environment'] = {'status': 'failed', 'error': str(e)}

In [ ]:
import time

metrics_log = {
    'timestamp': datetime.now().isoformat(),
    'phases': {}
}

# FASE 1: Generar datos
print("=" * 70)
print("FASE 1: GENERACIÓN DE DATOS")
print("=" * 70)

try:
    from data_gen import SyntheticTelemetry
    
    start = time.time()
    print(f"\n⏳ Generando 10 laps de datos sintéticos...")
    
    gen = SyntheticTelemetry()
    session = gen.generate_race_session(n_laps=10)
    
    elapsed = time.time() - start
    
    metrics_log['phases']['data_generation'] = {
        'duration_seconds': elapsed,
        'laps': session.metadata['num_laps'],
        'samples': len(session.telemetry_df),
        'mean_speed_kmh': session.metadata['mean_speed_kmh'],
        'max_speed_kmh': session.metadata['max_speed_kmh'],
        'mean_hr_bpm': session.metadata['mean_hr_bpm'],
        'status': 'success'
    }
    
    print(f"✅ Datos generados en {elapsed:.2f}s")
    print(f"   📊 {len(session.telemetry_df)} muestras")
    print(f"   🏃 Velocidad media: {session.metadata['mean_speed_kmh']:.1f} km/h")
    print(f"   💓 HR media: {session.metadata['mean_hr_bpm']:.1f} bpm")
    
except Exception as e:
    print(f"❌ Error en generación de datos: {e}")
    traceback.print_exc()
    metrics_log['phases']['data_generation'] = {'status': 'failed', 'error': str(e)}

## 3️⃣ Generar Datos y Entrenar Modelo

In [ ]:
sys.path.insert(0, str(SRC_DIR))

try:
    from config import (
        SIM_CONFIG, REWARD_CONFIG, TRAIN_CONFIG, VIS_CONFIG, PATHS, get_config_summary
    )
    
    print("✅ Configuración cargada desde config.py")
    
    config_summary = get_config_summary()
    config_df = pd.DataFrame(list(config_summary.items()), columns=['Parámetro', 'Valor'])
    print("\n📊 Resumen de Configuración:")
    print(config_df.to_string(index=False))
    
except Exception as e:
    print(f"⚠️ Error cargando configuración: {e}")
    SIM_CONFIG = None
    REWARD_CONFIG = None
    TRAIN_CONFIG = None

## 2️⃣ Cargar Configuración

In [ ]:
def validate_structure() -> Dict[str, bool]:
    """Validar estructura del proyecto"""
    status = {}
    
    required_dirs = {
        'src': SRC_DIR,
        'models': MODELS_DIR,
        'notebooks': NOTEBOOKS_DIR,
        'scripts': SCRIPTS_DIR,
        'logs': LOGS_DIR,
        'data': DATA_DIR,
    }
    
    required_files = {
        'config.py': SRC_DIR / 'config.py',
        'data_gen.py': SRC_DIR / 'data_gen.py',
        'environment.py': SRC_DIR / 'environment.py',
        'train.py': SRC_DIR / 'train.py',
        'visualize.py': SRC_DIR / 'visualize.py',
        'requirements.txt': PROJECT_ROOT / 'requirements.txt',
    }
    
    print("🔍 Validando directorios:")
    for name, path in required_dirs.items():
        exists = path.exists()
        status[f'dir_{name}'] = exists
        symbol = "✅" if exists else "❌"
        print(f"  {symbol} {name}: {path}")
    
    print("\n🔍 Validando archivos:")
    for name, path in required_files.items():
        exists = path.exists()
        status[f'file_{name}'] = exists
        symbol = "✅" if exists else "❌"
        print(f"  {symbol} {name}: {path}")
    
    all_ok = all(status.values())
    print(f"\n{'✅' if all_ok else '⚠️'} Estructura: {'OK' if all_ok else 'INCOMPLETA'}")
    
    return status

structure_status = validate_structure()

## 1️⃣ Validar Estructura del Proyecto

In [ ]:
import os
import sys
import json
import csv
import logging
import subprocess
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import traceback
import glob
import shutil

# Data and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.dates import DateFormatter

# Setup paths
PROJECT_ROOT = Path.cwd()
if (PROJECT_ROOT / "moto_bio_project").exists():
    PROJECT_ROOT = PROJECT_ROOT / "moto_bio_project"
    
SRC_DIR = PROJECT_ROOT / "src"
MODELS_DIR = PROJECT_ROOT / "models"
LOGS_DIR = PROJECT_ROOT / "logs"
DATA_DIR = PROJECT_ROOT / "data"
NOTEBOOKS_DIR = PROJECT_ROOT / "notebooks"
SCRIPTS_DIR = PROJECT_ROOT / "scripts"
METRICS_DIR = LOGS_DIR / "metrics"
ARTIFACTS_DIR = LOGS_DIR / "artifacts"

# Ensure directories exist
for d in [MODELS_DIR, LOGS_DIR, DATA_DIR, METRICS_DIR, ARTIFACTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler(LOGS_DIR / 'notebook_run.log')
    ]
)

print(f"✅ Entorno configurado")
print(f"📁 Raíz del proyecto: {PROJECT_ROOT}")
print(f"📁 Src: {SRC_DIR}")
print(f"📁 Modelos: {MODELS_DIR}")
print(f"📁 Logs: {LOGS_DIR}")

# 🏍️ Bio-Adaptive Haptic Coaching System
## Notebook de Análisis y Orquestación Completa

**Propósito**: Ejecutar, visualizar y documentar el deployment completo del sistema

- ✅ Validar estructura del proyecto
- ✅ Preparar dependencias
- ✅ Ejecutar training y evaluation
- ✅ Guardar métricas
- ✅ Visualizar resultados
- ✅ Generar reportes

**Fecha**: 2025-01-17 | **Versión**: 1.0